In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [8]:
req = Request('https://www.edmunds.com/mazda/3/2019/features-specs/', headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req)
bs = BeautifulSoup(webpage.read(),'html.parser')
print(bs)

<!DOCTYPE html>

<!-- 1.1.3056,N:P,V:P,E:P21 -->
<html class="" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="en-us" http-equiv="content-language"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<title data-react-helmet="true">2019 Mazda 3 Features &amp; Specs | Edmunds</title><meta content="2019 Mazda 3 Features &amp; Specs | Edmunds" data-react-helmet="true" name="title"><meta content="Detailed features and specs for the 2019 Mazda 3 including fuel economy, transmission, warranty, engine type, cylinders, drivetrain and more. Read reviews, browse our car inventory, and more." data-react-helmet="true" name="description"><meta content="96106612846" data-react-helmet="true" property="fb:app_id"><meta content="5798888989" data-react-helmet="true" property="fb:page_id"><meta content="Edmunds" data-react-helmet="true" property="og:site_name"/><meta content="6230134, 9700214, 300100577